# Importing the libraries

In [1]:
# Data Reading
from google.cloud import bigquery
import pandas as pd
import numpy as np

# Visualization
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'
import matplotlib.pyplot as plt

# Model Building
from sklearn.linear_model import LinearRegression

# Pickling the file
import pickle

# Writing Data
import pandas_gbq

In [13]:
!pip install pandas-gbq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.0/122.0 kB 1.8 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.34.0
    Uninstalling google-api-core-1.34.0:
      Successfully uninstalled google-api-core-1.34.0
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-python-client 1.8.0 requires google-api-core<2dev,>=1.13.0, but you have google-api-core 2.15.0 which is incompatible.


## Reading Data

In [2]:
client = bigquery.Client()

# Perform a query.
QUERY = (
    'SELECT * FROM `bigquery-public-data.sdoh_cdc_wonder_natality.county_natality` '
    'LIMIT 10')

query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

data = pd.DataFrame(data=[list(row.values()) for row in rows],
                  columns=[field.name for field in rows.schema])

# Display the DataFrame
display(data.head())

# Print the type of Dataframe
print(type(data))

,Year,County_of_Residence,County_of_Residence_FIPS,Births,Ave_Age_of_Mother,Ave_OE_Gestational_Age_Wks,Ave_LMP_Gestational_Age_Wks,Ave_Birth_Weight_gms,Ave_Pre_pregnancy_BMI,Ave_Number_of_Prenatal_Wks
0,2018-01-01,"Calhoun County, AL",01015,1265,26.67,38.11,38.25,3168.14,28.85,10.45
1,2018-01-01,"Tulsa County, OK",40143,8933,28.13,38.18,38.25,3243.39,27.58,11.04
2,2018-01-01,"Carroll County, GA",13045,1540,27.20,38.50,38.50,3253.35,27.98,10.75
3,2018-01-01,"Saginaw County, MI",26145,2182,27.41,38.75,38.50,3226.90,28.85,11.58
4,2018-01-01,"Hillsborough County, FL",12057,17126,28.97,38.41,38.50,3230.36,26.75,10.64


<class 'pandas.core.frame.DataFrame'>


# EDA

## Info

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Year                         10 non-null     object 
 1   County_of_Residence          10 non-null     object 
 2   County_of_Residence_FIPS     10 non-null     object 
 3   Births                       10 non-null     int64  
 4   Ave_Age_of_Mother            10 non-null     float64
 5   Ave_OE_Gestational_Age_Wks   10 non-null     float64
 6   Ave_LMP_Gestational_Age_Wks  10 non-null     float64
 7   Ave_Birth_Weight_gms         10 non-null     float64
 8   Ave_Pre_pregnancy_BMI        10 non-null     float64
 9   Ave_Number_of_Prenatal_Wks   10 non-null     float64
dtypes: float64(6), int64(1), object(3)
memory usage: 928.0+ bytes


## Value Counts

In [4]:
for col in data.columns.to_list():
    print("\n------- {0} -------\n".format(col))
    print(data[col].value_counts())


------- Year -------

Year
2018-01-01    10
Name: count, dtype: int64

------- County_of_Residence -------

County_of_Residence
Calhoun County, AL         1
Tulsa County, OK           1
Carroll County, GA         1
Saginaw County, MI         1
Hillsborough County, FL    1
Lake County, IN            1
St. Tammany Parish, LA     1
Osceola County, FL         1
Sarpy County, NE           1
Kane County, IL            1
Name: count, dtype: int64

------- County_of_Residence_FIPS -------

County_of_Residence_FIPS
01015    1
40143    1
13045    1
26145    1
12057    1
18089    1
22103    1
12097    1
31153    1
17089    1
Name: count, dtype: int64

------- Births -------

Births
1265     1
8933     1
1540     1
2182     1
17126    1
5785     1
2932     1
4437     1
2386     1
6337     1
Name: count, dtype: int64

------- Ave_Age_of_Mother -------

Ave_Age_of_Mother
26.67    1
28.13    1
27.20    1
27.41    1
28.97    1
28.12    1
29.12    1
28.64    1
29.43    1
29.63    1
Name: count, dtype:

## Nulll Values

In [5]:
for col in data.columns.to_list():
    print("\n{0} -------> {1}".format(col, round(data[col].isna().sum()/data.shape[0],2)))


Year -------> 0.0

County_of_Residence -------> 0.0

County_of_Residence_FIPS -------> 0.0

Births -------> 0.0

Ave_Age_of_Mother -------> 0.0

Ave_OE_Gestational_Age_Wks -------> 0.0

Ave_LMP_Gestational_Age_Wks -------> 0.0

Ave_Birth_Weight_gms -------> 0.0

Ave_Pre_pregnancy_BMI -------> 0.0

Ave_Number_of_Prenatal_Wks -------> 0.0


## Visualization

In [6]:
fig = px.box(data, y=["Births","Ave_Age_of_Mother","Ave_OE_Gestational_Age_Wks",
                                "Ave_LMP_Gestational_Age_Wks","Ave_Birth_Weight_gms","Ave_Pre_pregnancy_BMI",
                                "Ave_Number_of_Prenatal_Wks"],
             labels={
                     "variable": "Columns with numerical values",
                     "value": "Values",
                 },
             # template="plotly_dark",
             title="Outliers Visulaization")
fig.update_layout(title_x = 0.5,)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# Model Building

In [7]:
x = np.array([5, 15, 25, 35, 45, 55]).reshape((-1, 1))
y = np.array([5, 20, 14, 32, 22, 38])

model = LinearRegression()

model = LinearRegression().fit(x, y)
print("Model fitted")

r_sq = model.score(x, y)
print("coefficient of determination: ",r_sq)

print("intercept: ",model.intercept_)

print("slope: ",model.coef_)

Model fitted
coefficient of determination:  0.7158756137479542
intercept:  5.633333333333329
slope:  [0.54]


## Prediction

In [8]:
y_pred = model.predict(x)
print("Responses: ",y_pred)

Responses:  [ 8.33333333 13.73333333 19.13333333 24.53333333 29.93333333 35.33333333]


## Pickling

In [9]:
pickle.dump(model, open('model.pkl','wb'))

# Writing to Big Query

In [13]:
project_id = "clean-pen-405815" 
table_id = 'Viz.Sample'
object_cols = data.select_dtypes(include=['object']).columns

for column in object_cols:
    dtype = str(type(data[column].values[0]))
    if dtype == "<class 'datetime.date'>":
        data[column]  = pd.to_datetime(data[column] , infer_datetime_format=True)

# df.to_gbq(destination_table=PROCESSED_DATA_TABLE_NAME, if_exists='replace')
pandas_gbq.to_gbq(data, table_id, project_id,if_exists='replace')

/var/tmp/ipykernel_28762/1603484085.py:8: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

100%|██████████| 1/1 [00:00<00:00, 3043.76it/s]
